In [1]:
import os
import pandas as pd
from datetime import datetime,timedelta
import sqlite3
import xlwings as xw
import shutil
import tempfile
import glob
import re
from pypdf import PdfReader
import warnings
warnings.simplefilter(action='ignore', category=Warning)
excel_dir = "\\\\igcp.pt\\wss\\Import_Export\\Export\\PRD\\Dashboard_Export\\"
today = datetime.today()
days_to_subtract = 1 if today.weekday() > 0 else 3
yesterday = (today - timedelta(days=days_to_subtract)).strftime('%Y%m%d')

In [2]:
file_name = f"NCF-REPOS_{yesterday}_*.xlsx"
file_path_ = os.path.join(excel_dir, file_name)
file_paths = glob.glob(file_path_)
excel_file = pd.ExcelFile(file_paths[-1])
df_avaliacoes_table = pd.read_excel(excel_file, sheet_name="Avaliacoes", skiprows=1)

In [3]:
def excel_serial_to_date(serial):
    # Excel's date system starts on January 1, 1900, so we adjust accordingly
    excel_start_date = datetime(1900, 1, 1) - timedelta(days=2)  # Account for Excel's leap year bug
    return (excel_start_date + timedelta(days=serial)).strftime('%Y-%m-%d')


In [4]:
db_paths = r"\\igcp-fs\NCF-GER\Projetos 010-01-01\1_DESENVOLVIMENTO_PROJECTOS_INTERNOS\DASHBOARD_DIARIO\DB\Colateral_Repos.db"
conn = sqlite3.connect(db_paths)
cursor = conn.cursor()

# Carregar Avaliacões na Base de Dados 

In [5]:
if not df_avaliacoes_table.empty:
    # add today’s position date
    df_avaliacoes_table["Position_date"] = today.strftime("%Y-%m-%d")
    
    # rename columns
    df_avaliacoes_table = df_avaliacoes_table.rename(
        columns={
            "Number": "Transacao",
            "Counterparty": "Contraparte",
            "Opening Date": "Opening_date",
            "Value Date": "Value_date",
            "Expiry Date": "Expiry_date",
            "Security Identifier": "ISIN",
            "(IGCP) Start Cash": "Start_cash",
            "(IGCP) Coll. Clean Price": "Clean_price",
            "(IGCP) Coll. Dirty Price": "Dirty_price",
            "(IGCP) Accrued Interest": "Accrued_interest_cash",
            "(IGCP) Coll. Accrued Interest": "Accrued_interest_bond",
            "(IGCP) MTM": "MTM",
        }
    )

    # keep only the columns you want, in that order
    df_avaliacoes_table = df_avaliacoes_table[
        [
            "Transacao",
            "Contraparte",
            "Position_date",
            "Opening_date",
            "Value_date",
            "Expiry_date",
            "ISIN",
            "Start_cash",
            "Nominal",
            "Clean_price",
            "Dirty_price",
            "Accrued_interest_cash",
            "Accrued_interest_bond",
            "MTM",
        ]
    ]
    date_columns = ['Opening_date', 'Value_date', 'Expiry_date']

    
    for col in date_columns:
        df_avaliacoes_table[col] = df_avaliacoes_table[col].str.replace(',', '.').astype(float)
        df_avaliacoes_table[col] = df_avaliacoes_table[col].apply(excel_serial_to_date)
  

In [6]:
df_avaliacoes_table = df_avaliacoes_table.loc[df_avaliacoes_table["MTM"] != 0]

In [7]:
df_avaliacoes_table

Transacao Contraparte Position_date Opening_date  Value_date Expiry_date  \
0       167074    BBVA MAD    2025-09-09   2025-08-07  2025-08-11  2025-09-11   
1       167075    BBVA MAD    2025-09-09   2025-08-07  2025-08-11  2025-09-11   
2       167076    BBVA MAD    2025-09-09   2025-08-07  2025-08-11  2025-09-11   
3       167077    BBVA MAD    2025-09-09   2025-08-07  2025-08-11  2025-09-11   
4       167078    BBVA MAD    2025-09-09   2025-08-07  2025-08-11  2025-09-11   
..         ...         ...           ...          ...         ...         ...   
150     169519    BRED PAR    2025-09-09   2025-09-08  2025-09-09  2025-09-16   
151     169520    BRED PAR    2025-09-09   2025-09-08  2025-09-09  2025-09-16   
152     169521    BRED PAR    2025-09-09   2025-09-08  2025-09-09  2025-09-16   
153     169522    BRED PAR    2025-09-09   2025-09-08  2025-09-09  2025-09-16   
154     169523    BRED PAR    2025-09-09   2025-09-08  2025-09-09  2025-09-16   

             ISIN    Start_cash    Nominal  Clean_price  Dirty_price  \
0    AT0000A2CQD2  2.654115e+08  295000000      90.1480        90.15   
1    FR0013313582  2.039660e+08  238000000      84.8320        84.83   
2    FR0013516549  1.594113e+08  183000000      87.1270        87.13   
3    AT0000A2NW83  9.946500e+07  114000000      87.4325        87.43   
4    FR0013286192  1.068930e+08  111000000      96.2430        96.24   
..            ...           ...        ...          ...          ...   
150  EU000A3LVAL6  2.041740e+08  200000000       0.0000         0.00   
151  FR0013313582  1.699436e+08  200000000      84.8320        84.83   
152  FR0013313582  8.497180e+07  100000000      84.8320        84.83   
153  AT0000383864  9.532423e+07   87782000     107.6315       107.63   
154  FR0010916924  5.113389e+07   50000000     100.9475       100.95   

     Accrued_interest_cash  Accrued_interest_bond           MTM  
0              416917.2277           0.000000e+00 -1.081828e+05  
1              320396.5932           8.721233e+05  1.514113e+06  
2              250408.5826           0.000000e+00  2.192986e+05  
3              156242.9352           0.000000e+00 -5.180706e+04  
4              167911.0852           2.440479e+05 -1.286686e+04  
..                     ...                    ...           ...  
150             11257.9275           2.000000e+08  4.185258e+06  
151              9370.5013           7.328767e+05 -4.439062e+05  
152              4685.2506           3.664384e+05 -2.219531e+05  
153              5256.0721           8.417452e+05  6.656977e+03  
154              2819.4661           6.568493e+05  6.114001e+03  

[141 rows x 14 columns]

In [8]:
df_avaliacoes_table.to_sql(
        "avaliacoes_igcp_repos", conn, if_exists="append", index=False
    )

141

# Carregar Preços

In [9]:
df_precos_table = pd.read_excel(excel_file, sheet_name="Coll_prices", skiprows=1)

In [10]:
df_precos_table=df_precos_table.drop(columns={"Row Id"})

In [11]:
df_precos_table=df_precos_table.rename(columns = {"Spread Bid-Offer p.b":"Spread_Bid_Offer","Mid Price":"Mid_price","Security Identifier":"Security_identifier"})

In [12]:
df_precos_table["Date"] = df_precos_table["Date"].str.replace(',', '.').astype(float)
df_precos_table["Date"] = df_precos_table["Date"].apply(excel_serial_to_date)

In [13]:
df_precos_table

Date                       Name                  Item         Bid  \
0    2025-09-08               BT_16JAN2026        YS5438926 Corp   99.292000   
1    2025-09-08               BT_17JUL2026        YM2363294 Corp   98.305000   
2    2025-09-08               BT_19SEP2025        YV7804565 Corp   99.950000   
3    2025-09-08               BT_20MAR2026        YQ4026023 Corp   98.949000   
4    2025-09-08               BT_21NOV2025        YT6229868 Corp   99.605000   
..          ...                        ...                   ...         ...   
335  2025-09-08           OT_4.1_15APR2037        EF3288133 Corp  108.177000   
336  2025-09-08           OT_4.1_15FEB2045        EK6943477 Corp  104.790000   
337  2025-09-08         OT_4.125_14APR2027        AM1050047 Corp  103.377000   
338  2025-09-08             OTRV_18JUL2031  YM4493446 @EXCH Corp   99.240000   
339  2017-05-31  R-CONSOLIDADO_3_1942_2017                   NaN   78.961291   

            Ask  Spread_Bid_Offer   Mid_price Security_identifier  
0     99.305000               1.3   99.298500        PTPBTDGE0061  
1     98.334000               2.9   98.319500        PTPBTBGE0071  
2     99.952000               0.2   99.951000        PTPBTXGE0042  
3     98.968000               1.9   98.958500        PTPBTYGE0041  
4     99.612000               0.7   99.608500        PTPBTOGE0068  
..          ...               ...         ...                 ...  
335  108.335000              15.8  108.256000        PTOTE5OE0007  
336  105.010000              22.0  104.900000        PTOTEBOE0020  
337  103.444000               6.7  103.410500        PTOTEUOE0019  
338  100.490000             125.0   99.865000        PTOTVNOE0009  
339   78.961291               0.0   78.961291                 NaN  

[340 rows x 8 columns]

In [14]:
df_precos_table.to_sql(
        "preco_instrumentos_coll", conn, if_exists="append", index=False
    )

340